In [13]:
%cd /kaggle/working
# Clean re-clone to avoid stale files
!rm -rf Wan2GP
!git clone --depth 1 https://github.com/sanek1989/Wan2GP_MultiGPU.git
%cd Wan2GP_MultiGPU
# Ensure submodules are initialized (if any)
!git submodule update --init --recursive || echo "no submodules"
# Quick tree to verify structure
!find . -maxdepth 3 -type d -name "matanyone" -print || true
# Install dependencies compatible with Kaggle CUDA (skip strict torch pin)
!pip install --upgrade pip wheel setuptools --no-input --no-cache-dir --progress-bar off
!pip install -r requirements.txt --no-cache-dir --progress-bar off
# Ensure PyTorch exists (usually preinstalled on Kaggle); fallback if missing
import importlib
try:
    importlib.import_module("torch")
    import torch; print(f"Torch {torch.__version__}")
except Exception:
    # Kaggle often uses CUDA 12.1 images; adjust if needed
    !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121 --no-cache-dir --progress-bar off
# Extra deps used by the project if not in requirements
!pip install triton sageattention==1.0.6 --no-cache-dir --progress-bar off

/kaggle/working
fatal: destination path 'Wan2GP_MultiGPU' already exists and is not an empty directory.
fatal: destination path 'Wan2GP_MultiGPU' already exists and is not an empty directory.
/kaggle/working/Wan2GP_MultiGPU
/kaggle/working/Wan2GP_MultiGPU
./preprocessing/matanyone
./preprocessing/matanyone/matanyone
./preprocessing/matanyone
./preprocessing/matanyone/matanyone
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of opencv-python-headless to determine which version is compatible with other requirements. This could take a while.
  

In [19]:
# Ensure packages are importable as modules
import os
from pathlib import Path

base = Path('/kaggle/working/Wan2GP_MultiGPU')
for pkg in [base / 'preprocessing', base / 'preprocessing' / 'matanyone']:
    pkg.mkdir(parents=True, exist_ok=True)
    init_file = pkg / '__init__.py'
    if not init_file.exists():
        init_file.write_text('')
        print(f"Created {init_file}")
    else:
        print(f"Exists {init_file}")

print("✅ Ensured __init__.py in preprocessing and matanyone")


Exists /kaggle/working/Wan2GP_MultiGPU/preprocessing/__init__.py
Exists /kaggle/working/Wan2GP_MultiGPU/preprocessing/matanyone/__init__.py
✅ Ensured __init__.py in preprocessing and matanyone


In [20]:
# Replace 'TkAgg' with 'Agg' programmatically for headless Kaggle
file_path = "/kaggle/working/Wan2GP_MultiGPU/preprocessing/matanyone/tools/interact_tools.py"

from pathlib import Path
p = Path(file_path)
if p.exists():
    content = p.read_text()
    content = content.replace("matplotlib.use('TkAgg')", "matplotlib.use('Agg')")
    p.write_text(content)
    print("✅ Backend fixed to 'Agg' for headless environment.")
else:
    print(f"ℹ️ File not found: {file_path}. Skipping backend fix.")

✅ Backend fixed to 'Agg' for headless environment.


In [ ]:
# Kaggle blocks apt-get in most environments; avoid system packages.
# Use headless wheels to bypass libGL needs if OpenCV is used.
!pip install opencv-python-headless --no-cache-dir --progress-bar off
# ffmpeg is available on Kaggle; verify presence
!ffmpeg -version | head -n 1 | c

In [23]:
#Модели Wan2GP пробуем логировать и добовлять символические ссылки
!mkdir -p /kaggle/working/Wan2GP/ckpts

!ln -s /kaggle/input/wan2gp/wan2.1_image2video_480p_14B_quanto_mfp16_int8.safetensors \
      /kaggle/working/Wan2GP/ckpts/wan2.1_image2video_480p_14B_quanto_mfp16_int8.safetensors

!ln -s /kaggle/input/loras-wan2gp/Wan21_I2V_14B_lightx2v_cfg_step_distill_lora_rank64_fixed.safetensors \
      /kaggle/working/Wan2GP/loras_i2v/Wan21_I2V_14B_lightx2v_cfg_step_distill_lora_rank64_fixed.safetensors

ln: failed to create symbolic link '/kaggle/working/Wan2GP/ckpts/wan2.1_image2video_480p_14B_quanto_mfp16_int8.safetensors': File exists
ln: failed to create symbolic link '/kaggle/working/Wan2GP/loras_i2v/Wan21_I2V_14B_lightx2v_cfg_step_distill_lora_rank64_fixed.safetensors': No such file or directory
ln: failed to create symbolic link '/kaggle/working/Wan2GP/loras_i2v/Wan21_I2V_14B_lightx2v_cfg_step_distill_lora_rank64_fixed.safetensors': No such file or directory


In [24]:
# Ensure repo root and subpackages are on PYTHONPATH; try import but do not abort on failure
import os, sys, traceback, subprocess

repo_root = "/kaggle/working/Wan2GP_MultiGPU"
# change working dir if present
try:
    os.chdir(repo_root)
except Exception as e:
    print(f"Could not chdir to {repo_root}: {e}")

paths_to_add = [repo_root, os.path.join(repo_root, "preprocessing")]
for p in paths_to_add:
    if p not in sys.path:
        sys.path.insert(0, p)
print("SYS.PATH head:", sys.path[:3])

# Try to locate 'matanyone' directory to confirm expected structure
found = False
for root, dirs, files in os.walk(repo_root):
    if os.path.basename(root) == "matanyone":
        print("Found 'matanyone' at:", root)
        found = True
        break
if not found:
    print("'matanyone' directory not found under repo root (this may be OK if you use different layout)")

# Import check with full traceback (do not raise)
try:
    import preprocessing.matanyone as _mat
    print("OK: import preprocessing.matanyone ->", getattr(_mat, '__file__', '<unknown>'))
except Exception:
    print("Import warning: could not import preprocessing.matanyone")
    traceback.print_exc()

# Show CLI help without launching servers — run `python wgp.py --help` with PYTHONPATH set
env = os.environ.copy()
env['PYTHONPATH'] = ":".join([repo_root, os.path.join(repo_root, 'preprocessing')])
print('\nRunning: python wgp.py --help\n')
try:
    subprocess.run([sys.executable, 'wgp.py', '--help'], env=env, check=False)
except FileNotFoundError:
    print('wgp.py not found in repo root; ensure you cloned the repository to', repo_root)
except Exception as e:
    print('Failed to run wgp.py --help:', e)


SYS.PATH head: ['/kaggle/working/Wan2GP_MultiGPU/preprocessing', '/kaggle/working/Wan2GP_MultiGPU', '/kaggle/working/Wan2GP/preprocessing']
Found 'matanyone' at: /kaggle/working/Wan2GP_MultiGPU/preprocessing/matanyone
OK: import preprocessing.matanyone -> /kaggle/working/Wan2GP/preprocessing/matanyone/__init__.py

Running: python wgp.py --help



Error in sitecustomize; set PYTHONVERBOSE for traceback:
ModuleNotFoundError: No module named 'http.client'; 'http' is not a package


Error importing huggingface_hub.file_download: No module named 'http.client'; 'http' is not a package


Traceback (most recent call last):
  File "/kaggle/working/Wan2GP_MultiGPU/wgp.py", line 124, in <module>
    import gradio as gr
  File "/usr/local/lib/python3.11/dist-packages/gradio/__init__.py", line 3, in <module>
    import gradio._simple_templates
  File "/usr/local/lib/python3.11/dist-packages/gradio/_simple_templates/__init__.py", line 1, in <module>
    from .simpledropdown import SimpleDropdown
  File "/usr/local/lib/python3.11/dist-packages/gradio/_simple_templates/simpledropdown.py", line 7, in <module>
    from gradio.components.base import Component, FormComponent
  File "/usr/local/lib/python3.11/dist-packages/gradio/components/__init__.py", line 1, in <module>
    from gradio.components.annotated_image import AnnotatedImage
  File "/usr/local/lib/python3.11/dist-packages/gradio/components/annotated_image.py", line 8, in <module>
    import gradio_client.utils as client_utils
  File "/usr/local/lib/python3.11/dist-packages/gradio_client/__init__.py", line 1, in <module>